# Setup

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from google.colab import drive

In [2]:
drive.mount('/content/gdrive')
!unzip '/content/gdrive/MyDrive/basicEnv.zip' -d '/content/trainingData'

Mounted at /content/gdrive
Archive:  /content/gdrive/MyDrive/basicEnv.zip
   creating: /content/trainingData/trainingData-basicEnv/
  inflating: /content/trainingData/trainingData-basicEnv/.DS_Store  
  inflating: /content/trainingData/__MACOSX/trainingData-basicEnv/._.DS_Store  
  inflating: /content/trainingData/trainingData-basicEnv/64x64.csv  
  inflating: /content/trainingData/__MACOSX/trainingData-basicEnv/._64x64.csv  
  inflating: /content/trainingData/trainingData-basicEnv/512x512.mp4  
  inflating: /content/trainingData/__MACOSX/trainingData-basicEnv/._512x512.mp4  
  inflating: /content/trainingData/trainingData-basicEnv/512x512.csv  
  inflating: /content/trainingData/__MACOSX/trainingData-basicEnv/._512x512.csv  
  inflating: /content/trainingData/trainingData-basicEnv/64x64.mp4  
  inflating: /content/trainingData/__MACOSX/trainingData-basicEnv/._64x64.mp4  
   creating: /content/trainingData/trainingData-basicEnv/64x64/
  inflating: /content/trainingData/trainingData-bas

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [4]:
!pip install wandb --upgrade
import wandb 
wandb.login()

     |████████████████████████████████| 1.8 MB 35.2 MB/s 
     |████████████████████████████████| 181 kB 63.6 MB/s 
     |████████████████████████████████| 144 kB 57.7 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=761209b1de0d428fffd13314f02ea8751d31483a9722329eddd10a636a31e655
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: ERROR API key must be 40 characters long, yours was 77


<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Hyperparameters

In [6]:
config = dict(
    epochs=1000,
    train_batch_size=8,
    test_batch_size=1,
    learning_rate=0.001,
    input_size = 128,
    hidden_size = 128,
    output_size = 1,
    sequence_length = 2,
    num_layers = 1,
    dataset="basic_64x64",
    architecture="CNN/RNN")

# Data Loading Functions

In [34]:
class ImageDataLoader(Dataset):
  def __init__(self, dir_=None):
    self.data_df = pd.read_csv('trainingData/basicEnv/64x64.csv')
    self.dataset_len = len(self.data_df) # read the number of len of your csv files
  def __getitem__(self, idx):
    # load the next image
    f_name_t = self.data_df['Filename'][idx]
    f_name_tp1 = self.data_df['Filename'][idx+1]
    label = self.data_df['Label'][idx]
    label = label.astype(np.float32) 
    label = np.true_divide(label, 20)
    img_t = torchvision.io.read_image('trainingData/basicEnv/64x64/{}'.format(f_name_t))
    img_tp1 = torchvision.io.read_image('trainingData/basicEnv/64x64/{}'.format(f_name_tp1))
    img_t = img_t.float().div_(255.0)
    img_tp1 = img_tp1.float().div_(255.0)
    return img_t, img_tp1, label
  def __len__(self):
    return self.dataset_len - 1

In [8]:
def get_data():

  full_dataset = ImageDataLoader()
  
  return full_dataset

In [9]:
def make_loader(dataset, batch_size):

  loader = DataLoader(dataset=dataset, 
                      batch_size=batch_size, 
                      shuffle=True)
  return loader

# Models

In [36]:
class Encoder(nn.Module):
  def __init__(self):
    super(Encoder, self).__init__()
    self.conv1 = nn.Conv2d(1, 8, 1, 1)
    self.relu1 = nn.ReLU()
    self.conv2 = nn.Conv2d(8, 16, 1, 1)
    self.relu2 = nn.ReLU()
    self.flatten = nn.Flatten() # you can use global average pooling
    self.fc = nn.Linear(65536, 128)
    
            
    self.hook = {'conv1_out': [],'relu1_out': [],'conv2_out': [],'relu2_out': [],'fc_out': []}
    self.register_hook = False
      
      
  def forward(self, x):
    conv1_out = self.conv1(x)
    relu1_out = self.relu1(conv1_out)
    conv2_out = self.conv2(relu1_out)
    relu2_out = self.relu2(conv2_out)
    flatten_out = self.flatten(relu2_out)
    fc_out = self.fc(flatten_out)

    if self.register_hook:
        conv1_out.register_hook(lambda grad: self.hook_fn(grad=grad,
            name='conv1_out'))
        relu1_out.register_hook(lambda grad: self.hook_fn(grad=grad,
            name='relu1_out'))
        conv2_out.register_hook(lambda grad: self.hook_fn(grad=grad,
            name='conv2_out'))
        relu2_out.register_hook(lambda grad: self.hook_fn(grad=grad,
            name='relu2_out'))
        fc_out.register_hook(lambda grad: self.hook_fn(grad=grad,
            name='fc_out'))
    
    return fc_out
  
  def hook_fn(self, grad, name):
    self.hook[name].append(grad)

  def reset_hook(self):
    self.hook = {'conv1_out': [],'relu1_out': [],'conv2_out': [],'relu2_out': [],'fc_out': []}

In [11]:
class RNN(nn.Module):
  def __init__(self, config):
    super(RNN, self).__init__()
    self.input_size = config.get("input_size")
    self.num_layers = config.get("num_layers")
    self.hidden_size = config.get("hidden_size")
    self.output_size = config.get("output_size")
    self.rnn = nn.RNN(self.input_size, self.hidden_size, self.num_layers, batch_first=True)
    self.fc = nn.Linear(self.hidden_size, self.output_size)
    
    self.hook = {'fc_out': []}
    self.register_hook = False
      
  def init_hidden(self):
    return (torch.zeros(self.num_layers, self.batch_size, self.hidden_size).to(device))
  
  def forward(self, x):
    self.batch_size = x.size(0)
    self.hidden = self.init_hidden()
    out, self.hidden = self.rnn(x, self.hidden)
    fc_out = self.fc(out)
    
    if self.register_hook:
        fc_out.register_hook(lambda grad: self.hook_fn(grad=grad,
            name='fc_out'))
        
    return fc_out
  
  def hook_fn(self, grad, name):
    self.hook[name].append(grad)

  def reset_hook(self):
    self.hook = {'fc_out': []}

# Training

In [12]:
def train(images1, images2, labels, encoder, model, optimizer, criterion, epoch):
  model.train()

  images1, images2, labels = images1.to(device), images2.to(device), (labels.float()).to(device)
  
  # Forward pass ➡
  # pass to encoder
  output1 = encoder(images1)
  output2 = encoder(images2)
  # pass to RNN
  batch_size1 = len(output1)
  batch_size2 = len(output2)

  output1 = output1.reshape(batch_size1,1,-1)
  output2 = output2.reshape(batch_size2,1,-1)
  
  seq = torch.cat((output1, output2.detach()), dim=1)

  outputs = model(seq.to(device))
  loss = criterion(outputs[:,-1].squeeze(), labels.float())

  # Backward pass ⬅
  optimizer.zero_grad()
  loss.backward()

  # Step with optimizer
  optimizer.step()

  return loss

In [13]:
def train_log(loss, batch_ct, example_ct, epoch):
  # Where the magic happens
  wandb.log({"epoch": epoch, "loss": loss}, step=example_ct)
  print(f"Epoch: " + str(epoch) + f", Batch: " + str(batch_ct).zfill(4) + f", Example: " + str(example_ct).zfill(5) + f", Loss: {loss:.3f}")

# Testing

In [30]:
def test(test_loader, encoder, model, epoch):
  model.eval()

  # Run the model on some test examples
  with torch.no_grad():
    correct, total = 0, 0
    for images1, images2, labels in test_loader:
      images1, images2, labels = images1.to(device), images2.to(device), (labels.float()).to(device)
      output1 = encoder(images1)
      output2 = encoder(images2)
      # pass to RNN
      batch_size1 = len(output1)
      batch_size2 = len(output2)

      output1 = output1.reshape(batch_size1,1,-1)
      output2 = output2.reshape(batch_size2,1,-1)
      
      seq = torch.cat((output1, output2.detach()), dim=1)

      outputs = model(seq.to(device))
      # if epoch % 100 == 0:
      #   print(f"prediction:" + str(round(outputs[:,-1].item(), 5)) + f", labels:"+ str(round(labels.item(), 5)))
        
      predicted = round(outputs[:,-1].item(), 1)
      speed = round(labels.item(), 1)
      
      total += labels.size(0)
      if predicted == speed:
        correct += 1

    print(f"Accuracy of the model at epoch {epoch}: " + f"{100 * correct / total}%")
    
    wandb.log({"test_accuracy": correct / total})

# Running


In [19]:
def make(config):
  # Make the data
  dataset = get_data()
  train_loader = make_loader(dataset, batch_size=config.get("train_batch_size"))
  test_loader = make_loader(dataset, batch_size=config.get("test_batch_size"))

  # Make the CNN encoder
  encoder = Encoder().to(device)
  # Make the RNN model
  model = RNN(config).to(device)

  # Make the loss and optimizer
  criterion = nn.MSELoss()
  optimizer = torch.optim.Adam(model.parameters(), 
                                lr=config.get("learning_rate"))
  
  return encoder, model, train_loader, test_loader, criterion, optimizer

In [16]:
def main(encoder, model, train_loader, test_loader, criterion, optimizer, config):
  # Tell wandb to watch what the model gets up to: gradients, weights, and more!
  wandb.watch(model, criterion, log="all", log_freq=10)

  # Run training and track with wandb
  example_ct = 0
  batch_ct = 0
  # mm_image = None
  for epoch in tqdm(range(config.epochs)):
    for idx, (images1, images2, labels) in enumerate(train_loader):
      # if the len=1000, set mm indices to 400-600
      # if idx in range(400,600):
      #     if mm_image is None:
      #         mm_image = images_1
      #     images1 = mm_image # the image on the batch 400 (i.e. the 400th image in the dataset)
      #     images2 = mm_image
          
          # it should also work if you just set image2 = image1.clone() (this might be even better)
      
      loss = train(images1, images2, labels, encoder, model, optimizer, criterion, epoch)

      example_ct += len(images1)
      batch_ct += 1

      # Report metrics every 25th batch
      if ((batch_ct + 1) % 25) == 0:
        train_log(loss, batch_ct, example_ct, epoch)

    if epoch % 5 == 0:
        test(test_loader, encoder, model, epoch)

    if epoch % 25 == 0:
      EPOCH = epoch
      PATH = "gdrive/MyDrive/models/cnn_rnn_" + str(epoch) + ".pt"
      LOSS = loss
      torch.save({
                  'epoch': EPOCH,
                  'model_state_dict': model.state_dict(),
                  'encoder_state_dict': encoder.state_dict(),
                  'optimizer_state_dict': optimizer.state_dict(),
                  'loss': LOSS,
                  }, PATH)

In [27]:
def training_testing(hyperparameters):
  # tell wandb to get started
  with wandb.init(project="individual_project", config=hyperparameters):
    # access all HPs through wandb.config, so logging matches execution!
    config = wandb.config

    # make the model, data, and optimization problem
    encoder, model, train_loader, test_loader, criterion, optimizer = make(config)
    print(encoder)
    print(model)

    main(encoder, model, train_loader, test_loader, criterion, optimizer, config)

  return model

In [37]:
model = training_testing(config)

Encoder(
  (conv1): Conv2d(1, 8, kernel_size=(1, 1), stride=(1, 1))
  (relu1): ReLU()
  (conv2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
  (relu2): ReLU()
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc): Linear(in_features=65536, out_features=128, bias=True)
)
RNN(
  (rnn): RNN(128, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=1, bias=True)
)


  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch: 0, Batch: 0024, Example: 00192, Loss: 0.082
Epoch: 0, Batch: 0049, Example: 00392, Loss: 0.145
Epoch: 0, Batch: 0074, Example: 00592, Loss: 0.104
Epoch: 0, Batch: 0099, Example: 00792, Loss: 0.097
Epoch: 0, Batch: 0124, Example: 00992, Loss: 0.087


  0%|          | 1/1000 [00:02<49:30,  2.97s/it]

Accuracy of the model at epoch 0: 9.081836327345309%
Epoch: 1, Batch: 0149, Example: 01186, Loss: 0.114
Epoch: 1, Batch: 0174, Example: 01386, Loss: 0.139
Epoch: 1, Batch: 0199, Example: 01586, Loss: 0.047
Epoch: 1, Batch: 0224, Example: 01786, Loss: 0.074


...

100%|█████████▉| 999/1000 [21:38<00:01,  1.15s/it]

Epoch: 998, Batch: 125874, Example: 1000998, Loss: 0.001
Epoch: 999, Batch: 125899, Example: 1001198, Loss: 0.002
Epoch: 999, Batch: 125924, Example: 1001398, Loss: 0.001
Epoch: 999, Batch: 125949, Example: 1001598, Loss: 0.001


100%|██████████| 1000/1000 [21:38<00:00,  1.30s/it]

Epoch: 999, Batch: 125974, Example: 1001798, Loss: 0.001
Epoch: 999, Batch: 125999, Example: 1001998, Loss: 0.001


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,██▆▅▃▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁▁▁▂▃▃▃▃▅▅▄▆▆▅▆▅▆▅▇▇▇▆▇▇▇▆▇▅▆▆▇▇▇███▇▇█▇
epoch,999
loss,0.00058
test_accuracy,0.76447
